In [1]:
import os
import json
import requests
from dotenv import load_dotenv
from openai import OpenAI
from loguru import logger
from IPython.display import display, Markdown
from PyPDF2 import PdfReader
import gradio as gr
from types import SimpleNamespace

load_dotenv(override=True)

openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    logger.error("OPENAI_API_KEY is not set in the environment variables.")
else:
    logger.info(f"OpenAI API exists: {openai_api_key[:10]}...")

openai_client = OpenAI(api_key=openai_api_key)

/Users/ashiq/PycharmProjects/AIAgentDevelopment/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-10 22:58:04.331 | INFO     | __main__:<module>:18 - OpenAI API exists: sk-proj-Yq...


In [2]:
pushover_user = os.getenv("PUSHOVER_USER_KEY")
pushover_token = os.getenv("PUSHOVER_API_KEY")
pushover_url = "https://api.pushover.net/1/messages.json"
if not pushover_user or not pushover_token:
    logger.error("PUSHOVER_USER_KEY or PUSHOVER_API_KEY is not set in the environment variables.")
else:
    logger.info(f"Pushover User exists: {pushover_user[:10]}...")
    logger.info(f"Pushover Token exists: {pushover_token[:10]}...")

2025-08-10 22:58:04.346 | INFO     | __main__:<module>:7 - Pushover User exists: uxdm8c1jfb...
2025-08-10 22:58:04.346 | INFO     | __main__:<module>:8 - Pushover Token exists: ao2yz9nvk4...


In [3]:
def send_pushover_notification(message):
    if not pushover_user or not pushover_token:
        logger.error("Pushover user or token is not set.")
        return

    payload = {
        "user": pushover_user,
        "token": pushover_token,
        "message": message
    }

    try:
        response = requests.post(pushover_url, data=payload)
        response.raise_for_status()
        logger.info("Pushover notification sent successfully.")
    except requests.RequestException as e:
        logger.error(f"Failed to send Pushover notification: {e}")

In [4]:
send_pushover_notification("hi there, this is a test message from the Agentic AI Engineering notebook.")

2025-08-10 22:58:05.870 | INFO     | __main__:send_pushover_notification:15 - Pushover notification sent successfully.


In [5]:
def record_user_details(name, email, notes):
    send_pushover_notification(f"Recording {name} with email {email} and notes: {notes}")
    return f"User {name} recorded with email {email} and notes: {notes}."

In [6]:
def record_unknown_question(question):
    send_pushover_notification(f"Recording unknown question: {question}")
    return f"Unknown question recorded: {question}."

In [7]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "use this tool to record that a user is interested in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The name of the user.",
            },
            "email": {
                "type": "string",
                "description": "The email address of the user.",
            },
            "notes": {
                "type": "string",
                "description": "Additional notes about the user.",
            },
        },
        "required": ["email"],
        "additionalProperties": False,
    },
}

In [8]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "use this tool to record that a user asked an unknown question",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The unknown question asked by the user.",
            },
        },
        "required": ["question"],
        "additionalProperties": False,
    },
}

In [9]:
tools = [
    {
        "type": "function",
        "function": record_user_details_json,
    },
    {
        "type": "function",
        "function": record_unknown_question_json,
    }
]

In [10]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'use this tool to record that a user is interested in touch and provided an email address',
   'parameters': {'type': 'object',
    'properties': {'name': {'type': 'string',
      'description': 'The name of the user.'},
     'email': {'type': 'string',
      'description': 'The email address of the user.'},
     'notes': {'type': 'string',
      'description': 'Additional notes about the user.'}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': 'use this tool to record that a user asked an unknown question',
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': 'The unknown question asked by the user.'}},
    'required': ['question'],
    'additionalProperties': False}}}]

In [11]:
def handle_tool_call(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)

        results.append({
            "role": "tool",
            "content": json.dumps(result),
            "tool_call_id": tool_call.id,
        })
    return results

In [12]:
mock_tool_call = SimpleNamespace(function=SimpleNamespace(name="record_user_details",
                                                          arguments='{"name": "John", "email": "john@test.com", "notes": "test user"}'),
                                 id="test_id_1")
handle_tool_call([mock_tool_call])

Tool called: record_user_details


2025-08-10 22:58:08.838 | INFO     | __main__:send_pushover_notification:15 - Pushover notification sent successfully.


[{'role': 'tool',
  'content': '"User John recorded with email john@test.com and notes: test user."',
  'tool_call_id': 'test_id_1'}]

In [13]:
# Dynamic way to handle tool calls
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else None
        results.append({
            "role": "tool",
            "content": json.dumps(result),
            "tool_call_id": tool_call.id,
        })
    return results

In [14]:
mock_tool_call2 = SimpleNamespace(
    function=SimpleNamespace(name="record_unknown_question", arguments='{"question": "What is AI?"}'), id="test_id_2")
handle_tool_calls([mock_tool_call2])

Tool called: record_unknown_question


2025-08-10 22:58:09.551 | INFO     | __main__:send_pushover_notification:15 - Pushover notification sent successfully.


[{'role': 'tool',
  'content': '"Unknown question recorded: What is AI?."',
  'tool_call_id': 'test_id_2'}]

In [15]:
reader = PdfReader("me/Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

    with open("me/about.txt", "r", encoding="utf-8") as f:
        summary = f.read()

name = "Gazi Ashiq"

In [16]:
system_prompt = f"""You are acting as {name}. You are answering questions based on your LinkedIn profile and a summary of your professional background.
Your LinkedIn profile is as follows:
{linkedin}
Your summary is as follows:
{summary}
You are a professional in the field of AI and software engineering, with a focus on building AI
applications and web development. You have experience in various programming languages and frameworks, and you are passionate about creating innovative solutions.
You are friendly, professional, and concise in your responses. Always provide accurate information based on the
provided LinkedIn profile and summary. If you don't know the answer, say "I don't know" instead of making up an answer.
If you are dont know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if you are not sure about the question.
If the user is engaging in discussion, try to starer the towards getting in touch by email; ask for their email address and use the record_user_details tool to record their interest in getting in touch.
"""

In [17]:
def chat(message, history):
    messags = [
        {"role": "system", "content": system_prompt},
        *history,
        {"role": "user", "content": message}
    ]
    done = False
    while not done:
        try:
            response = openai_client.chat.completions.create(
                model="gpt-5-mini",
                messages=messags,
                tools=tools,
            )

            finish_reason = response.choices[0].finish_reason

            if finish_reason == "tool_calls":
                message = response.choices[0].message
                tool_calls = message.tool_calls
                results = handle_tool_calls(tool_calls)
                messags.append(message)
                messags.extend(results)
            else:
                done = True
                return response.choices[0].message.content
        except Exception as e:
            logger.error(f"Error in chat function: {e}")
            return "An error occurred while processing your request. Please try again later."

In [18]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
